<a href="https://colab.research.google.com/github/syphaxAouadene/Cours_programmation_concurrente/blob/main/my_first_neural_network_for_handwritten_digit_classification_version_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import time
from IPython.display import clear_output
import json
import matplotlib.pyplot as plt
%pylab inline
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
from mlxtend.data import loadlocal_mnist
import platform

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# def fcl(inputs_layer, nbr_neurons, weights, biais, activation_type='ReLU'):
#     current_layer = []
#     current_layer = flatten(np.dot(weights, inputs_layer) + biais) 
#     output_layer = activation_function(current_layer, activation_type)
#     return output_layer.reshape(len(output_layer), 1)

# def fcl_softmax(inputs_layer, nbr_neurons, weights, biais, activation_type='ReLU'):
#     current_layer = []
#     current_layer = flatten(np.dot(weights, inputs_layer) + biais)
#     output_layer = current_layer
#     return output_layer.reshape(len(output_layer), 1)
   
def activation_function(layer, type_of_activation='relu'):
    type_of_activation = type_of_activation.lower()
    switcher = {
        'relu': ReLU,
        'tanh': tanh,
        'segmoid': segmoid
    }
    # Get the function from switcher dictionary
    activation_type = switcher.get(type_of_activation, lambda: "Invalid type_of_activation_function, please choose either 'ReLU' or 'tanh' or 'segmoid' !")
    return activation_type(layer)
    
    
def ReLU(layer):
    return layer * (layer > 0)

def d_ReLU(layer):
    return 1. * (layer > 0)

def tanh(layer):
    r = (np.exp(layer)-np.exp(-1*layer))/(np.exp(layer)+np.exp(-1*layer))   
    return np.array(r)

def d_tanh(layer):
    return 1 - tanh(layer) * tanh(layer)

def segmoid(layer):
    return np.array(1/(1+np.exp(-1*layer)))

def d_segmoid(vector):
    """
    cette fontion prend un vector en entrée et retourne la dérivée de segmoid par rapport a ce vector
    """
    return segmoid(vector) * (1 - segmoid(vector))

def softmax(data):
    proba_values = np.exp(data)/(np.sum(np.exp(data)))   
    return np.array(proba_values)

def categoricalCrossEntropy(generated_values, target_values):
    somme = 0
    for i in range(len(generated_values)):
        somme = somme + target_values[i] * np.log(generated_values[i])
    return (-1) * somme 


# def update_weights(dL_dY, weights, biais, inputs, lrate):
#     """
#     arguments :
#     dL_dY : un vecteur des dérivées de la couche supérieure par rapport a la couche de sortie Y de dimension n_outputs
#     weights : la matrice des poids de dimension (n_inputs x n_outputs)
#     inputs : le vecteur de sortie de la couche précedente de dimension n_inputs
#     lrate : learning rate (scalar)
#     """
#     dL_dW = np.dot(dL_dY, np.transpose(inputs))
#     new_weights = flatten(weights) - lrate * flatten(dL_dW)
#     new_biais = biais - lrate * dL_dY
#     return new_weights.reshape(weights.shape), new_biais

# def calcul_dL_dX(dL_dY, weights):
#     return np.dot(np.transpose(weights), dL_dY)

def normelize(img):
    return img/255

def flatten(img):
    return img.flatten()

def show_image(img):
    plt.imshow(img, cmap=plt.cm.binary)
    plt.show()

# def normelize_with_mean_and_std(img):
#     return (img - img.mean())/(img.std())
    
def init_params(my_network):
    nbr_layers = len(my_network) - 1
#     W, B = np.array([]), np.array([])
    W, B = [], []
    for i in range(nbr_layers):
        W.append(np.random.randn(my_network[i+1], my_network[i]))
        B.append(np.random.randn(my_network[i+1], 1))
#         np.concatenate([W, np.random.randn(my_network[i+1], my_network[i])])
#         np.concatenate([B, np.random.randn(my_network[i+1], 1)])
    return W, B

def forward_pass(img, W, B):
    """
    here we will use this notation :
    Z[i] = W[i].X + B[i]
    A[i] = activation_function(Z[i])
    Z is a list that carries all the output of each layer
    A is a list that carries all the output of each activation function
    """
    Z, A = [], [img]
    for i in range(len(W)):
        if i == 0: # we have to multiply input layer with weights because we're in the first layer
            Z.append(np.dot(W[i], img) + B[i])
            A.append(ReLU(Z[i]))
        elif i == len(W)-1: # we have to use softmax as activation layer because we're in the last layer
            Z.append(np.dot(W[i], A[i]) + B[i])
            A.append(softmax(Z[i]))
        else: # we're in the hidden layer
            Z.append(np.dot(W[i], A[i]) + B[i])
            A.append(ReLU(Z[i]))
    return Z, A

def one_hot(y):
    return np.eye(10)[y].reshape(10, 1)

def update_W_and_B(W, dL_dW, B, dL_dB, lr):
    """
    this function update the weights and Biais of myNetwork
    arguments : 
    - W : it is a list that contains each Weight vector ([W1, W2, ...])
    - dL_dW : derivatives of loss with respect to Weights (it is a list that contains Weights derivatives vectors [dL_dW1, dL_dW2, ...])
    - B : it is a list that contains each Biais vector ([B1, B2, ...])
    - dL_dB : derivatives of loss with respect to Biais (it is a list that contains Biais derivatives vectors [dL_dB1, dL_dB2, ...])
    - lr : learning rate (real number)
    """
    new_W = []
    new_B = []
    for w, dw in zip(W, dL_dW):
#         print(w.shape, dw.shape)
        w = w - lr * dw
        new_W.append(w)
    for b, db in zip(B, dL_dB):
        b = b - lr * db
        new_B.append(b)
    return new_W, new_B

def compute_accuracy(x_val, y_val, W, B):
    '''
        This function does a forward pass of x_validation, then checks if the indices
        of the maximum value in the output equals the indices in the label
        y. Then it sums over each prediction and calculates the accuracy.
    '''
    predictions = []

    for x, y in zip(x_val, y_val):
        # prepare the input image
        X = flatten(x)
        X = X.reshape(len(X), 1)
        Y = one_hot(y)
        # forward-propagation
        Z, A = forward_pass(X, W, B)
        output = A[-1]
        pred = np.argmax(output)
        predictions.append(pred == np.argmax(Y))

    return np.mean(predictions)

def classify(img, W, B):
    """
    cette fonction recois une seule image en parametre
    et elle reçois les poids W et les Biais B
    et elle retourne la catégorie de l'image en entier 0..9
    """
    predictions = []
    X = flatten(img)
    X = X.reshape(len(X), 1)
    # forward-propagation
    Z, A = forward_pass(X, W, B)
    output = A[-1]
    pred = np.argmax(output)
    return pred

def show_accuracies(train_images, train_labels, val_images, val_labels, test_images, test_labels, W, B):
    """
    this function compute accuracy for each train-set, validation-set, and test-set
    then print them all.
    arguments : train_images, train_labels, val_images, val_labels, test_images, test_labels, W, B
    """
    train_accuracy = compute_accuracy(train_images, train_labels, W, B)
    val_accuracy = compute_accuracy(val_images, val_labels, W, B)
    test_accuracy = compute_accuracy(test_images, test_labels, W, B)
    print("Accuracies :\n\
    - train accuracy = {} %\n\
    - val accuracy = {} %\n\
    - test accuracy = {} %".format(train_accuracy*100, val_accuracy*100, test_accuracy*100))

def backpro_pass(dL_dZ, A, Z, W, indice):
    dl_dw = np.dot(dL_dZ, np.transpose(A[indice]))
    dl_db = dL_dZ
    dl_dz = 0
    if indice*(-1) != len(Z)+1:
      dl_da = np.dot(np.transpose(W[indice+1]), dL_dZ)
      da_dz = d_ReLU(Z[indice])
      dl_dz = dl_da * da_dz
    return dl_dw, dl_db, dl_dz

In [5]:
images_path = '/content/drive/MyDrive/Colab Notebooks/mnist_data/train-images.idx3-ubyte'
labels_path = '/content/drive/MyDrive/Colab Notebooks/mnist_data/train-labels.idx1-ubyte'
test_images_path = '/content/drive/MyDrive/Colab Notebooks/mnist_data/test-images.idx3-ubyte'
test_labels_path = '/content/drive/MyDrive/Colab Notebooks/mnist_data/test-labels.idx1-ubyte'
test_images, test_labels = loadlocal_mnist(test_images_path, test_labels_path)
train_images, train_labels = loadlocal_mnist(images_path, labels_path)
# group all the images in one list
# then normelize all the images
images = np.concatenate([train_images, test_images])
labels = np.concatenate([train_labels, test_labels])
images = normelize(images)
# shuffle all the images and all labels randomly
random.seed(12)
indices = np.arange(len(labels))
np.random.shuffle(indices)
labels = labels[indices]
images = images[indices]
# change shape of the images
images = images.reshape(len(images), 28, 28)
# split the data into train, validation and test 
train_images, val_images, test_images = images[:60000], images[60000:65000], images[65000:]
train_labels, val_labels, test_labels = labels[:60000], labels[60000:65000], labels[65000:]

In [ ]:
# Initialize my network
my_network = [28*28, 128, 10]
number_epochs = 50
# Initialize weights and biais of my_network
W, B = init_params(my_network)

# training
start_time = time.time()
losses = []
accuracies = []
for epoch in range(number_epochs):
    epoch_losses = np.array([])
    start_epoch_time = time.time()
    for i in range(len(train_labels)):
        # prepare the input image
        X = flatten(train_images[i])
        X = X.reshape(len(X), 1)
        Y = one_hot(train_labels[i])
        # forward-propagation
        Z, A = forward_pass(X, W, B)
        loss = categoricalCrossEntropy(A[-1], Y)
        epoch_losses = np.concatenate([epoch_losses, loss])
# example of neural network architecture :
        # 28*28 ---> 128        ---> 20         ---> 10
        # A[0] ----> Z[0], A[1] ---> Z[1], A[2] ---> Z[2], A[3]
        # ---------> W[0], B[0] ---> W[1], B[1] ---> W[2], B[2]

        # backpropagation
        dL_dZ2 = A[-1] - Y
        dL_dW, dL_dB, dL_dZ = [], [], [dL_dZ2]

        indice = 0
        for layer in range(len(my_network)-1):
          dl_dw, dl_db, dl_dz = backpro_pass(dL_dZ[-1], A, Z, W, indice - 2)
          dL_dW.append(dl_dw)
          dL_dB.append(dl_db)
          dL_dZ.append(dl_dz)
          indice = indice - 1 
          
        # update weights W and Biais B  
        dL_dW.reverse()
        dL_dB.reverse()
        W, B = update_W_and_B(W, dL_dW, B, dL_dB, 0.01)
      
    # Test my model at epoch = gama    
    if (epoch % 5 == 0):
        accuracy = compute_accuracy(val_images, val_labels, W, B)
        accuracies.append(accuracy)
        print('-------------------------------------------------------------------------> Accuracy : ',accuracies[-1])

    print('epoch ',epoch,' -------> loss : ',np.array(epoch_losses).mean(), ' | time : ',(time.time() - start_epoch_time))
    losses.append(np.array(epoch_losses).mean())

print("--- %s seconds ---" % (time.time() - start_time), ' | time : ',(time.time() - start_time)) 
fig, ax = plt.subplots(2)
fig.suptitle('Graph of accuracy and loss')
ax[0].plot(losses)
ax[1].plot(accuracies)

In [ ]:
show_accuracies(train_images, train_labels, val_images, val_labels, test_images, test_labels, W, B)